In [1]:
import pickle
import time
import os
import datetime
import numpy as np
import pandas as pd

cur_dir = os.getcwd()
main_dir = os.path.dirname(os.path.dirname(cur_dir))

In [2]:
test_dates = ["08_02_2022"]
test_folders = ["Day2_Training1_B_training"]
rtdstr_filenames = ["training_rtdstr"]
pzt_filenames = ["WTRUN2_day2_training1_B_2022-08-02_17-25-43-99_pzt_ch3F00FF00_0.05V"]
file_counts = [7]

eds_rtdstr_offsets_dir = os.path.join(cur_dir, "offset_pickles")

In [3]:
#Verified that SG/RTD and PZT data exactly overlap at initiation. 
#So there's no need to offset a dataset from each other.

In [5]:
#The block to load in and arrange the PZT data.
pzt_combined_dfs = list()

# Read in the offsets in IMGenie data. We'll use these values to crop out any extra lines lingering in PZT data.
with open(os.path.join(eds_rtdstr_offsets_dir,'eds_rtdstr_offsets.pkl'), 'rb') as f:
  eds_rtdstr_offsets = pickle.load(f)
with open(os.path.join(eds_rtdstr_offsets_dir,'rtdstr_offsets_trailing.pkl'), 'rb') as f:
  rtdstr_offsets_trailing = pickle.load(f)


for i in range(len(test_folders)):
  pzt_onetest_dfs = list()
  data_dir = os.path.join(main_dir, test_dates[i]+"_Tests", "testdata", test_folders[i])
  
  #Read the RTDSTR file in case we'll need to do some realignment
  rtdstr_filename = rtdstr_filenames[i]
  rtdstr_file_csv = os.path.join(data_dir, rtdstr_filename)
  rtdstr_file_df = pd.read_csv(rtdstr_file_csv+".csv", header=0)

  #Insert proper time object
  month = int (test_dates[i][0:2])
  day = int (test_dates[i][3:5])
  year = int (test_dates[i][6:])
  rtdstr_times = rtdstr_file_df["Date/Time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d_%H-%M-%S-%f"))
  rtdstr_times = rtdstr_times.apply(lambda x: datetime.datetime(year, month, day, x.hour, x.minute, x.second, x.microsecond)) #We do this because we haven't recorded date in the original data.
  
  if "rtdstr_DateTime Obj" not in rtdstr_file_df.columns:
    rtdstr_file_df.insert(1, "rtdstr_DateTime Obj", rtdstr_times)
  else:
    rtdstr_file_df["rtdstr_DateTime Obj"] = rtdstr_file_df["rtdstr_DateTime Obj"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))

  for j in range(1,file_counts[i]+1):
    print (f"Frocessing PZT file: {j}")
    filename = pzt_filenames[i]+"_"+str(j)+".csv"
    file_csv = os.path.join(data_dir, filename)
    file_df = pd.read_csv(file_csv, header=0)
    
    if j == 1: #When its the start of the run
      if eds_rtdstr_offsets[i][0] != "rtdstr": #if vantage[i] is not rtdstr (meaning that rtdstr started earlier, so some of it was cut).
        #Determine how many seconds of IMGenie data we cut when aligning the data and crop out that many lines from PZT data.
        seconds_passed = rtdstr_file_df["rtdstr_DateTime Obj"][eds_rtdstr_offsets[i][1]] - rtdstr_file_df["rtdstr_DateTime Obj"][0]
        pzt_lines_passed = seconds_passed.seconds * 10000 + int(seconds_passed.microseconds * 0.01) #10000 PZT samples per second.
        file_df = file_df.iloc[pzt_lines_passed:]
    
    if j == file_counts[i]: #When its the end of the run
      #Determine how many seconds of IMGenie data we cut when aligning the data and crop out that many lines from PZT data.
      seconds_passed = rtdstr_file_df["rtdstr_DateTime Obj"].iloc[-1] - rtdstr_file_df["rtdstr_DateTime Obj"].iloc[rtdstr_offsets_trailing[i]]
      if seconds_passed != datetime.timedelta(0): #if vantage[i] is not rtdstr (meaning that rtdstr ended later, so some of it was cut).
        pzt_lines_passed = seconds_passed.seconds * 10000 + int(seconds_passed.microseconds * 0.01) #10000 PZT samples per second.
        file_df = file_df.iloc[:-1*pzt_lines_passed] 
    pzt_onetest_dfs.append(file_df)
  pzt_combined_dfs.append(pzt_onetest_dfs)

Frocessing PZT file: 1
Frocessing PZT file: 2
Frocessing PZT file: 3
Frocessing PZT file: 4
Frocessing PZT file: 5
Frocessing PZT file: 6
Frocessing PZT file: 7


In [6]:
#Concat PZT dfs and save:
df_save_dir = os.path.join(main_dir, "ConsolidatedData", "Training1_B")
concated_pzt_dfs = list()

for i, test_dfs in enumerate(pzt_combined_dfs):
  concated_pzt_df = pd.concat((test_df for test_df in test_dfs), ignore_index=True)
  concated_pzt_df.to_pickle(os.path.join(df_save_dir,f'consolidated_pzt_{i}.pkl'))
  concated_pzt_dfs.append(concated_pzt_df)
concated_all_pzt_df = pd.concat((concated_pzt_df for concated_pzt_df in concated_pzt_dfs), ignore_index=True)
concated_all_pzt_df.to_pickle(os.path.join(df_save_dir,f'consolidated_pzt_all.pkl'))